## Note: In basic-pipeline code file, I code the neural network, sigmoid function, loss function,

## update parameters from scratch.

## Now, we'll use nn module to speedup and optimize the code.

## With the help of nn modules, we can create neural network easily and efficiently. Like torch.sigmoid, torch.optim etc.


In [3]:
import torch
import torch.nn as nn

In [4]:
# Assume, we are creating binary classification. In input layer, we have 5 features i.e weights, and using sigmoid activation function and there is one output.

class Model(nn.Module):
    
    # constructor
    def __init__(self, num_features):

        # super().__init__() is use to access the parent functions. For example: nn.Module is parent, Class model is child. I want to access the all functions even constructors of parent
        # function which is nn.Module. When I write this, it also means to register my custom model in nn.Module and with the help of this, I can detect parameters and optimizer will run
        # successfully.
        # WARNING: If you don't write this, you may not have fully access to parent functions, and can not register your model, can not detect parameters as well, which cause the optimizer
        # to fail
        super().__init__()

        # Let's build neural network with the help of nn modules
        # Here, nn.Linear means fully connected layer. For example: if num_feature is 5 i.e 5 features (input numbers) and then output size is 1.
        # NOTE: Here, we define the model architecture (layers and neurons).
        # No computation happens here — we are only setting up the structure.

        self.linear = nn.Linear(num_features, 1)
        self.activation = nn.Sigmoid()


    # Here, The forward() method defines how data passes through the model.

    def forward(self, features):

        # Here, features is the full input tensor (rows and columns), e.g., shape (10,5),
        # not just the number of features. The linear layer computes output = x*W + b,
        # resulting in shape (10,1) after multiplying (10,5) with weights of shape (5,1).
        output = self.linear(features)

        # Apply the activation function (sigmoid).
        # This keeps the same shape (10,1) but converts values to range [0,1].output = self.activation(output)
        return output

In [15]:
# Create a dummy dataset with 10 samples and 5 features
data = torch.randn([10, 5])

# Create a model object.
# data.shape[1] gives the number of features (columns) to define input size.
model = Model(data.shape[1])

# Pass data through the model to get output probabilities.
# You can use model(data) instead of model.forward(data),
# because PyTorch automatically calls forward().
# NOTE: It's best to use model(data), because, In industry, they follow this model(data), not model.feature(data)
output = model(data)
print(output)


tensor([[0.5347],
        [0.4857],
        [0.5416],
        [0.2007],
        [0.2530],
        [0.3960],
        [0.5203],
        [0.2011],
        [0.4441],
        [0.2977]], grad_fn=<SigmoidBackward0>)


In [17]:
# You can see model weights and bias as well.
model.linear.weight
model.linear.bias

Parameter containing:
tensor([-0.3241], requires_grad=True)

# If you want to visualize the model parameters, then you can use torchinfo for this.

In [20]:
from torchinfo import summary

summary(model, input_size=data.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Did you notice, above neural network was for 1 neuron.
# Now, let's create a bit complex neural network i.e hidden layer with number of neurons

In [26]:
# Assume, we are creating a binary classification model with one hidden layer.
# Input layer has 5 features means 5 neurons, hidden layer has 3 neurons,
# and output layer has 1 neuron with sigmoid activation function.

class complex_Model(nn.Module):

    # Constructor
    def __init__(self, num_features):

        # super().__init__() is used to access the parent class nn.Module.
        # This registers your custom model in PyTorch so that parameters can be tracked,
        # and optimizer can update them during training.
        super().__init__()

        # First fully connected (linear) layer: input features -> 3 hidden neurons
        # nn.Linear creates weights and bias for this layer.
        # NOTE: This only defines the layer architecture, no computation happens here.
        self.linear1 = nn.Linear(num_features, 3)

        # Activation function for hidden layer
        # ReLU introduces non-linearity to the network.
        self.ReLU = nn.ReLU()

        # Second fully connected (linear) layer: 3 hidden neurons -> 1 output
        self.linear2 = nn.Linear(3, 1)

        # Activation function for output layer
        # Sigmoid converts output to probability (range 0 to 1) for binary classification.
        self.sigmoid = nn.Sigmoid()

    # Forward pass: defines how input data flows through the network
    def forward(self, features):

        # Pass input through first linear layer
        # features is the full input tensor (rows = samples, columns = features), e.g., shape (10,5)
        # Linear layer computes output = x*W + b
        # Output shape after this layer: (10,3)
        output = self.linear1(features)

        # Apply ReLU activation to hidden layer
        # Keeps the shape (10,3) but introduces non-linearity
        output = self.ReLU(output)

        # Pass through second linear layer
        # Computes weighted sum of hidden neurons for the output layer
        # Output shape: (10,1)
        output = self.linear2(output)

        # Apply sigmoid activation for binary classification
        # Output shape remains (10,1), values between 0 and 1
        output = self.sigmoid(output)

        return output


In [37]:
new_data = torch.randn([10, 5])
model1 = complex_Model(new_data.shape[1])

model1(data)

tensor([[0.4867],
        [0.4831],
        [0.4852],
        [0.5157],
        [0.5152],
        [0.4852],
        [0.4800],
        [0.5213],
        [0.4856],
        [0.4908]], grad_fn=<SigmoidBackward0>)

In [39]:
model1.linear1.weight

Parameter containing:
tensor([[ 0.4351,  0.2339, -0.1694, -0.1105,  0.3747],
        [ 0.3863, -0.0105,  0.3144, -0.0650, -0.1686],
        [-0.2292,  0.1392, -0.0284,  0.3103,  0.2661]], requires_grad=True)

In [41]:
model1.linear2.weight

Parameter containing:
tensor([[ 0.0984, -0.0749,  0.1057]], requires_grad=True)

In [43]:
model1.linear2.bias

Parameter containing:
tensor([-0.0593], requires_grad=True)

In [35]:
summary(model1, new_data.shape)

Layer (type:depth-idx)                   Output Shape              Param #
complex_Model                            [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# If we want to build a neural network with many hidden layers (e.g., 10+)
# and each layer has 10–50 neurons, writing each layer manually is hard and repetitive.
# To simplify this, we can use nn.Sequential to build the network easily and make it reusable.
# The code of simplified version is given below:

In [44]:
# Assume we are building a binary classification model with one hidden layer.
# Input has 'num_features', hidden layer has 3 neurons, and output has 1 neuron with sigmoid activation.

class complex_Model(nn.Module):

    # Constructor
    def __init__(self, num_features):

        # Call parent class constructor to register model parameters
        super().__init__()

        # Using nn.Sequential to define the network in a simple and reusable way
        # nn.Sequential allows us to chain layers in the order they are applied
        # Here:
        # 1. Linear layer: input -> 3 hidden neurons
        # 2. ReLU activation
        # 3. Linear layer: 3 hidden neurons -> 1 output
        # 4. Sigmoid activation to output probability
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),  # input layer -> hidden layer
            nn.ReLU(),                    # activation for hidden layer
            nn.Linear(3, 1),              # hidden layer -> output layer
            nn.Sigmoid()                  # output activation (binary classification)
        )

    # Forward pass
    def forward(self, features):
        # features is the input tensor (batch_size x num_features), e.g., (10,5)
        # Sequential automatically passes input through all layers in order
        output = self.network(features)
        return output


In [45]:
model2 = complex_Model(new_data.shape[1])
model2(data)

tensor([[0.5062],
        [0.6025],
        [0.7929],
        [0.5657],
        [0.5628],
        [0.6278],
        [0.6883],
        [0.5118],
        [0.7380],
        [0.6997]], grad_fn=<SigmoidBackward0>)